In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import regularizers, Sequential, layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [2]:
import numpy as np
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import random

In [9]:
df = pd.read_csv('../speech_emotion_reco/data/merged_dataset.csv')
df['savee'] = df['path'].apply(lambda x: 1 if 'savee' in x else 0)
df['crema'] = df['path'].apply(lambda x: 1 if 'crema' in x else 0)
df['ravdess'] = df['path'].apply(lambda x: 1 if 'ravdess' in x else 0)
df['tess'] = df['path'].apply(lambda x: 1 if 'tess' in x else 0)

In [10]:
df.emotion.value_counts()

happy       1924
disgust     1923
sad         1923
angry       1923
fear        1923
neutral     1895
surprise     452
unknown      200
Name: emotion, dtype: int64

In [11]:
df[df['emotion']=='unknown']

,Unnamed: 0,gender,emotion,path,duration,savee,crema,ravdess,tess
3520,3520,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,1.821451,0,0,0,1
3521,3521,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,2.187574,0,0,0,1
3522,3522,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,2.157234,0,0,0,1
3523,3523,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,1.719410,0,0,0,1
3524,3524,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,2.153878,0,0,0,1
3525,3525,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,2.234240,0,0,0,1
3526,3526,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,1.954467,0,0,0,1
3527,3527,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,2.029932,0,0,0,1
3528,3528,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,2.145669,0,0,0,1
3529,3529,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,1.881723,0,0,0,1


In [17]:
def load_data():
    data_path = '../speech_emotion_reco/data/images_v2/'
    classes = {'happy': 0, 
               'sad': 1,
               'fear': 2,
               'disgust': 3,
               'angry': 4,
               'neutral': 5,
               'surprise': 6,
               'unknown': 7}
    imgs = []
    labels = []
    images_path = [os.path.join(data_path, path) for path in os.listdir(data_path) if path.find('.png') > 0]
    index = 0
    for path in images_path:
        if classes[df.loc[index, 'emotion']] in [0,1,2,3,4,5]:
            image = Image.open(path).convert('RGB')
            resized_image = image.resize((256,256))
            imgs.append(np.array(resized_image))
            labels.append(classes[df.loc[index, 'emotion']])
        index += 1
        
    X = np.array(imgs)
    num_classes = len(set(labels))
    y = to_categorical(labels, num_classes)

    return X, y

In [18]:
X, y = load_data()

In [19]:
X.shape

(11511, 256, 256, 3)

In [9]:
y.shape

(11511, 6)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size= 0.16, test_size=0.04, random_state=69)

In [21]:
X_train.shape

(1841, 256, 256, 3)

In [22]:
X_test.shape

(461, 256, 256, 3)

In [23]:
def load_model():
    model = Sequential()
    model.add(layers.Conv2D(120, kernel_size=11, activation="relu", strides=4, input_shape=(256,256,3)))
    model.add(layers.MaxPooling2D(3, strides=2))
    model.add(layers.Conv2D(256, kernel_size=5, activation="relu", strides=1))
    model.add(layers.Conv2D(384, kernel_size=3, activation="relu"))
    model.add(layers.Flatten())
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(6, activation='softmax'))
    
    return model

In [24]:
def build_model():  
    model = load_model()
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [ ]:
model = build_model()

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, verbose=1, restore_best_weights=True)

history = model.fit(X_train, y_train, 
                    validation_split=0.2, 
                    epochs=50, 
                    batch_size=16, 
                    callbacks=[es])